In [1]:
import requests
import time
import os
import re
import regex as rehttp://localhost:8888/notebooks/Scraper_Linkedin_user.ipynb#
import pandas as pd

from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
import selenium.webdriver as webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class Datos_linkedin:
  def __init__(self, url, puesto, empresa, lugar, logo, lapso_capturado):
    self.url = url
    self.puesto = puesto
    self.empresa = empresa
    self.lugar = lugar
    self.logo = logo
    self.lapso_capturado=lapso_capturado

In [2]:
#Accede al webdriver en pantalla completa

url = 'https://www.linkedin.com/jobs/jobs-in-mexico?trk=homepage-basic_guest_nav_menu_jobs&position=1&pageNum=0'
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
#options.add_argument('auto-open-devtools-for-tabs')
options.add_argument("--start-maximized")

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
driver.implicitly_wait(10)

C:\Users\Tole 01\AppData\Local\Temp\ipykernel_56284\502970832.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [3]:
boton_signin_top = driver.find_elements("xpath",'//a[contains(@data-tracking-control-name,"public_jobs_nav-header-signin")]')
boton_signin_top[0].click()
time.sleep(2)

input_username = driver.find_elements("xpath",'//input[contains(@id,"username")]')
input_password = driver.find_elements("xpath",'//input[contains(@id,"password")]')
boton_signin = driver.find_elements("xpath",'//button[contains(@class,"btn__primary--large from__button--floating")]')

input_username[0].send_keys('alawood1995@gmail.com')
time.sleep(1)
input_password[0].send_keys('alwoca95')
time.sleep(1)
boton_signin[0].click()


In [4]:
# Rellena las boxes principales del trabajo que buscan y la locacion en la que estan

driver.implicitly_wait(2)
input_trabajo = driver.find_elements("xpath",'//input[contains(@aria-label,"Busca por cargo, aptitud o empresa")]')
input_locacion = driver.find_elements("xpath",'//input[contains(@aria-label,"Ciudad, provincia/estado o código postal")]')

xpath_boton_buscar = """//button[contains(@class,"jobs-search-box__submit-button
      artdeco-button artdeco-button--2 artdeco-button--secondary")]"""

boton_buscar = driver.find_elements("xpath",xpath_boton_buscar)

input_trabajo[0].clear()
input_locacion[0].clear()

time.sleep(1)
input_trabajo[0].send_keys('Full Stack')
time.sleep(1)
input_locacion[0].send_keys('Estados Unidos')
time.sleep(1)
boton_buscar[0].click()
time.sleep(2)

In [5]:
# Delay del objeto WebDriverWait
delay = 20

# Activa la seccion de todos los filtros
boton_TodoslosFiltros = driver.find_elements("xpath",'//button[contains(@aria-label,"Mostrar todos los filtros. Al hacer clic en este botón, se muestran todas las opciones de filtros disponibles.")]')
boton_TodoslosFiltros[0].click()
time.sleep(1)

# Ubica los checkbox de 1 semana y remote
checkbox_1semana = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//input[@name="fecha-de-publicación-filter-value" and @value="r604800"]')))
checkbox_remoto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//input[@id="advanced-filter-workplaceType-2"]')))

# Focus en checkbox y rellena
checkbox_1semana.send_keys('space')
time.sleep(1)
checkbox_1semana.send_keys('Full Stack')
time.sleep(1)
checkbox_remoto.send_keys('space')
time.sleep(1)
checkbox_remoto.send_keys('Full Stack')
time.sleep(1)

# Ubica el boton de busqueda y lo clickea
boton_mostrar_resultados = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//div[@class="artdeco-modal__actionbar ember-view"]/div/button[@class="reusable-search-filters-buttons search-reusables__secondary-filters-show-results-button artdeco-button artdeco-button--2 artdeco-button--primary ember-view"]')))
boton_mostrar_resultados.click()

In [ ]:
# Boton para cerrar mensajes
time.sleep(1)
boton_cerrar_inbox = driver.find_element('xpath', '//button[contains(@class,"msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view")]/following-sibling::button')
boton_cerrar_inbox.click()

# Obtiene el numero total de vacantes
VacantesPorScrapear = driver.find_element('xpath', '//div[contains(@class,"jobs-search-results-list__title-heading")]/small').text
VacantesPorScrapear
num_vacantes = re.findall(r'\d+(?=[+,])?', VacantesPorScrapear)
num_vacantes=int(''.join(num_vacantes))
print(num_vacantes)

# Crea una lista de objetos de scraping
Objetos_linkedin = []

In [6]:
def scrollea_vacantes():
    # Scrollea por cachos con un tiempo minimo de scrolleo, para recopilar todas las tarjetas de las vacantes.
    class_del_scroll = """//div[contains(@class, "jobs-search-results-list
    
    ")]"""
    
    scroll_cards = driver.find_element('xpath',class_del_scroll)
    for x in range(20):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',scroll_cards)
        time.sleep(0.1)
    time.sleep(1)

def captura_y_srapea():
    # Crea lista interna de la funcion para hacer append
    nuevos_objetos_linkedin = [] 
    
    # Captura las tarjetas disponibles en el scroll
    class_card_clickable = """//li[contains(@class,"ember-view   jobs-search-results__list-item occludable-update p0 relative scaffold-layout__list-item
              
              
              ")]"""
    
    card_clickable = driver.find_elements("xpath",class_card_clickable)
    len(card_clickable)
    
    # Inicia el scraping
    for i in range(0,len(card_clickable)):
        try:
            # Clickea la tarjeta
            card_clickable[i].click()
            time.sleep(0.1)                                                                            
            
            # Encuentra los datos en la card original, por lo que los datos se encuentran en listas
            URL_logo = driver.find_elements('xpath','//a[@class="disabled ember-view job-card-container__link"]/img')
            URL_vacante = driver.find_elements("xpath",'//div[@class="mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view"]/a')
    
            # Encuentra los datos en las diferentes partes de la card que abre
            class_tiempo = """//span[@class="jobs-unified-top-card__posted-date
                  jobs-unified-top-card__posted-date--new t-bold" or @class="jobs-unified-top-card__posted-date
                  "]"""
            Timelapse_not_transformed = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",class_tiempo)))
            Puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//h2[@class="t-24 t-bold jobs-unified-top-card__job-title"]')))
            Empresa = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="ember-view t-black t-normal"]')))
            Lugar = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="jobs-unified-top-card__bullet"]')))
        
            try:   
                # Pasa los datos a distintos objetos, hace un append en una lista 
                nuevos_objetos_linkedin.append(Datos_linkedin(URL_vacante[i].get_attribute('href'), Puesto.text, Empresa.text, Lugar.text, URL_logo[i].get_attribute('src'), Timelapse_not_transformed.text)) 
                print(f'Ya obtuve el dato {i}')
    
            except:
                # Si no entra en la direccion de la tarjeta, entonces imprime esto
                print(f'No pude obtener el dato {i} dentro del objeto')
    
        except:
            # Si no encuentra la tarjeta, imprime esto y se pasa a la otra 
            print(f'No pude obtener el dato {i} por su direccion o porque no cargo la tarjeta')
            pass
    
    # Escoge el hermano del boton de la pagina que esta activo
    boton_Sig_Pag = WebDriverWait(driver, 2).until(EC.presence_of_element_located(('xpath','//li[@class="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view"]/following-sibling::li')))
    boton_Sig_Pag.click()
    return nuevos_objetos_linkedin

In [ ]:
pagina_scrapeada = 0

while pagina_scrapeada <= 40:
    scrollea_vacantes()
    lista_captura_y_srapea = captura_y_srapea()
    print(lista_captura_y_srapea)
    Objetos_linkedin.append(lista_captura_y_srapea)
    pagina_scrapeada += 1

Ya obtuve el dato 0
Ya obtuve el dato 1
Ya obtuve el dato 2
Ya obtuve el dato 3
Ya obtuve el dato 4
Ya obtuve el dato 5
Ya obtuve el dato 6
Ya obtuve el dato 7
Ya obtuve el dato 8
Ya obtuve el dato 9
Ya obtuve el dato 10
Ya obtuve el dato 11
Ya obtuve el dato 12
Ya obtuve el dato 13
Ya obtuve el dato 14
Ya obtuve el dato 15
Ya obtuve el dato 16
Ya obtuve el dato 17
Ya obtuve el dato 18
Ya obtuve el dato 19
Ya obtuve el dato 20
Ya obtuve el dato 21


In [ ]:
# Imprime los datos de cada objeto
print(len(Objetos_linkedin))
for J in range (len(Objetos_linkedin)):
    print(f'Lista numero {J}')
    for j in range(len(Objetos_linkedin[J])):
        print(f'Objeto numero {j}')
        print(Objetos_linkedin[J][j].url+'\n'+Objetos_linkedin[J][j].puesto+'\n'+Objetos_linkedin[J][j].empresa+'\n'+Objetos_linkedin[J][j].lugar+'\n'+Objetos_linkedin[J][j].logo+'\n'+Objetos_linkedin[J][j].lapso_capturado+'\n'+'----------')

In [ ]:
scroll_cards = driver.find_element('xpath',class_del_scroll)

In [ ]:
print('Fueron 40 paginas scrapeadas en 12 min 25 segundos')
print(f'Numero total de vacantes obtenidas fue de {25*40}' + '\n'+'---------')
lista_con_vacantes_totales = []

# Imprime los datos de cada objeto
print(len(Objetos_linkedin))
for J in range (len(Objetos_linkedin)):
    print(f'Lista numero {J} tiene un numero de objetos de: {len(Objetos_linkedin[J])}')
    lista_con_vacantes_totales.append(len(Objetos_linkedin[J]))
print('\n'+'-------'+'\n'+f'El numero de vacantes totales es de {sum(lista_con_vacantes_totales)}')
print(f'El % de eficiencia fue de {100*((sum(lista_con_vacantes_totales))/(25*40))}%')

In [ ]:
# CODIGO BASE PARA SCROLLEO Y SCRAPING DE INFORMACION

# Scrollea por cachos con un tiempo minimo de scrolleo, para recopilar todas las tarjetas de las vacantes.
class_del_scroll = """//div[contains(@class, "jobs-search-results-list
    
    ")]"""

scroll_cards = driver.find_element('xpath',class_del_scroll)
for x in range(10):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',scroll_cards)
    time.sleep(0.1)
time.sleep(1)

# Captura las tarjetas disponibles y se cambia a la nueva pagina para extraer info, luego regresa
class_card_clickable = """//li[contains(@class,"ember-view   jobs-search-results__list-item occludable-update p0 relative scaffold-layout__list-item
              
              
              ")]"""

card_clickable = driver.find_elements("xpath",class_card_clickable)
len(card_clickable)

# Inicia el scraping
for i in range(0,len(card_clickable)):
    try:
        # Clickea la tarjeta
        card_clickable[i].click()
        time.sleep(0.1)                                                                            
        
        # Encuentra los datos en la card original, por lo que los datos se encuentran en listas
        URL_logo = driver.find_elements('xpath','//a[@class="disabled ember-view job-card-container__link"]/img')
        URL_vacante = driver.find_elements("xpath",'//div[@class="mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view"]/a')

        # Encuentra los datos en las diferentes partes de la card que abre
        class_tiempo = """//span[@class="jobs-unified-top-card__posted-date
                  jobs-unified-top-card__posted-date--new t-bold" or @class="jobs-unified-top-card__posted-date
                  "]"""

        Timelapse_not_transformed = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",class_tiempo)))
        Puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//h2[@class="t-24 t-bold jobs-unified-top-card__job-title"]')))
        Empresa = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="ember-view t-black t-normal"]')))
        Lugar = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="jobs-unified-top-card__bullet"]')))
    
        try:   
            # Pasa los datos a distintos objetos, hace un append en una lista 
            Objetos_linkedin.append(Datos_linkedin(URL_vacante[i].get_attribute('href'), Puesto.text, Empresa.text, Lugar.text, URL_logo[i].get_attribute('src'), Timelapse_not_transformed.text)) 
            print(f'Ya obtuve el dato {i}')

        except:
            # Si no entra en la direccion de la tarjeta, entonces imprime esto
            print(f'No pude obtener el dato {i} dentro del objeto')

    except:
        # Si no encuentra la tarjeta, imprime esto y se pasa a la otra 
        print(f'No pude obtener el dato {i} por su direccion o porque no cargo la tarjeta')
        pass

# Escoge el hermano del boton de la pagina que esta activo
boton_Sig_Pag = WebDriverWait(driver, 2).until(EC.presence_of_element_located(('xpath','//li[@class="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view"]/following-sibling::li')))
boton_Sig_Pag.click()

In [ ]:
#DIRECCIONES DE LOS DATOS (Falta modalidad, salario y descripcion)

# URL de la vacante en la card original
URL_vacante = driver.find_elements("xpath",'//div[@class="mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view"]/a')
URL_vacante[i].get_attribute('href')

# El puesto al que aplican se encuentra en la card que se abre
Puesto = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//h2[@class="t-24 t-bold jobs-unified-top-card__job-title"]')))
Puesto.text

# La empresa a la que pertenece el puesto se encuentra en la card que se abre
Empresa = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//a[@class="ember-view t-black t-normal"]')))
Empresa.text


# El tiempo que ha transcurrido desde su publicacion. Este tiene que ser tratado posteriormente para que concuerde con la fecha actual
class_tiempo = """//span[@class="jobs-unified-top-card__posted-date
          jobs-unified-top-card__posted-date--new t-bold" or @class="jobs-unified-top-card__posted-date
          "]"""
Timelapse_not_transformed = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",class_tiempo)))
Timelapse_not_transformed.text

# Lugar de trabajo se encuentra en la card que se abre
Lugar = WebDriverWait(driver, delay).until(EC.presence_of_element_located(("xpath",'//span[@class="jobs-unified-top-card__bullet"]')))
Lugar.text

# URL del logo en la card original
URL_logo = driver.find_elements('xpath','//a[@class="disabled ember-view job-card-container__link"]/img')
URL_logo[i].get_attribute('src')


# SE TIENEN QUE ENCONTRAR ESTOS DATOS
# Son datos que contienen{ 0:Seniority, 1:Si el empleo es de tiempo completo o no, etc , 2:La funcion principal del trabajo}
Datos_extra = driver.find_elements("xpath",'//span[@class="description__job-criteria-text description__job-criteria-text--criteria"]')
Seniority = Datos_extra[0].text
Tipo_empleo = Datos_extra[1].text
Funcion_principal = Datos_extra[2].text


In [ ]:
# Scrollea todo la ventana de vacantes. El problema es que el scrolleo debe ser lento para que detecte todas las vacantes.
class_del_scroll = """//div[contains(@class, "jobs-search-results-list
    
    ")]"""

scroll_cards = driver.find_element('xpath',class_del_scroll)
driver.execute_script("arguments[0].scrollTo(0,3300);",scroll_cards)

In [ ]:
# print de las direcciones dentro de las tarjetas
for i in range(len(card_clickable)):
    card_clickable[i].click()
    print(i)
    print(Puesto.text +'\n'+Empresa.text+'\n'+Lugar.text+'\n'+Timelapse_not_transformed.text+'\n'+URL_logo[i].get_attribute('src')+'\n'+URL_vacante[i].get_attribute('href')+'\n'+'----------')